# Agent Manager with Azure Cosmos DB Metadata Storage

This notebook demonstrates how to use the enhanced AgentManager with Azure Cosmos DB for storing agent metadata.

## Prerequisites

- Azure AI Foundry project created
- Azure OpenAI resource connected
- Azure Cosmos DB account created
- Credentials configured in `.env.local`
- `agent_utils.py` and `agent_db.py` in the utils directory

## What You'll Learn

- ✅ Store agent metadata in Azure Cosmos DB alongside Azure Agent Service
- ✅ Rich metadata: categories, scenarios, maintainers, sample prompts
- ✅ Search and filter agents by status, category
- ✅ Export/import agent metadata
- ✅ Database statistics and querying
- ✅ AAD authentication with DefaultAzureCredential


## Table of Contents

1. [Setup and Installation](#setup-and-installation)
2. [Initialize Project Client and AgentManager](#initialize-project-client-and-agentmanager)
3. [Example 1: Create Agent with Extended Metadata](#example-1-create-agent-with-extended-metadata)
4. [Example 2: Retrieve Agent Metadata from Cosmos DB](#example-2-retrieve-agent-metadata-from-cosmos-db)
5. [Example 3: Update Agent and Metadata](#example-3-update-agent-and-metadata)
6. [Example 4: Query and Filter Agents](#example-4-query-and-filter-agents)
7. [Example 5: Search Agents](#example-5-search-agents)
8. [Example 6: Database Statistics](#example-6-database-statistics)
9. [Example 7: Export and Import Agent Metadata](#example-7-export-and-import-agent-metadata)
11. [Cleanup: Delete All Resources](#cleanup-delete-all-resources)
12. [Summary](#summary)


## Configure PATH for Azure CLI

Ensure the Azure CLI is accessible in the notebook kernel's PATH.

In [ ]:
import os
import shutil

# Replace with the directory you want to add
new_path_entry = "/opt/homebrew/bin"
current_path = os.environ.get('PATH', '')

if new_path_entry not in current_path.split(os.pathsep):
    os.environ['PATH'] = new_path_entry + os.pathsep + current_path
    print(f"Updated PATH for this session: {os.environ['PATH']}")
else:
    print(f"PATH already contains {new_path_entry}: {current_path}")

# You can then verify with shutil.which again
print(f"Location of 'az' found by kernel now: {shutil.which('az')}")

## Setup and Installation


In [ ]:
from agent_utils import AgentManager
from agent_db import AgentDB
import os
import sys
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

# Add utils directory to path
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'utils'))

# Load environment variables
load_dotenv("../.env")

print("✅ Imports successful")

## Initialize Project Client and AgentManager


In [ ]:
# Get project endpoint
endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
model = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o")

if not endpoint:
    raise ValueError("Please set AZURE_AI_PROJECT_ENDPOINT in .env")

# Verify Cosmos DB endpoint
cosmos_endpoint = os.getenv("AZURE_COSMOS_ENDPOINT")
if not cosmos_endpoint:
    raise ValueError("Please set AZURE_COSMOS_ENDPOINT in .env")

# Initialize client
project_client = AIProjectClient(
    endpoint=endpoint,
    credential=DefaultAzureCredential()
)

# Initialize AgentManager with Cosmos DB support
agent_manager = AgentManager(
    project_client=project_client
)

print("✅ Azure AI Project Client initialized")
print("✅ AgentManager with Cosmos DB initialized")
print(f"📦 Project Endpoint: {endpoint}")
print(f"📦 Cosmos DB Endpoint: {cosmos_endpoint}")
print(f"🤖 Model: {model}")

## Example 1: Create Agent with Extended Metadata


In [ ]:
import json
from azure.ai.agents.models import FunctionTool, ToolSet

# Define mock functions that will be used by the agent


def toolset_bing(query: str) -> str:
    """Search the web using Bing to find current information and research data"""
    # Mock implementation - in production this would call actual Bing API
    mock_results = {
        "query": query,
        "results": [
            {
                "title": f"Research result for: {query}",
                "url": "https://example.com/research",
                "snippet": f"Comprehensive information about {query} including market trends and analysis."
            }
        ]
    }
    return json.dumps(mock_results)


def research_analyzer(topic: str, data_points: str) -> str:
    """Analyze research data and provide structured insights on market trends and competitors"""
    # Mock implementation - in production this would perform actual analysis
    analysis = {
        "topic": topic,
        "key_findings": [
            "Market shows 30% growth in this sector",
            "Top 3 competitors identified",
            "User pain points documented"
        ],
        "recommendations": [
            "Focus on feature differentiation",
            "Consider pricing strategy adjustments"
        ],
        "data_quality": "high"
    }
    return json.dumps(analysis)


# Create FunctionTool with the mock functions
function_tool = FunctionTool(functions={toolset_bing, research_analyzer})

# Create toolset
toolset = ToolSet()
toolset.add(function_tool)

# Map function names to implementations for execution
functions_map = {
    "toolset_bing": toolset_bing,
    "research_analyzer": research_analyzer
}

print("✅ Mock functions defined")
print("🛠️  Available functions:")
print("  - toolset_bing: Search web for current information")
print("  - research_analyzer: Analyze research data and provide insights")

In [ ]:
# Create an agent with rich metadata AND actual function calling
agent = agent_manager.create_agent(
    model=model,
    name="Product Research Assistant",
    instructions="""## Role
You are a Product Research Assistant designed to help product managers and teams gather market insights,
analyze competitor products, and synthesize research findings. You have access to web search capabilities
and can help structure research findings into actionable insights.

## Constraints
1. Use clear and structured language in your replies, presented in Markdown format;
2. When researching competitors or market trends, use the "toolset_bing" function to gather current information;
3. Use the "research_analyzer" function to analyze data and provide structured insights;
4. Organize research findings into clear categories: market trends, competitor analysis, user needs, and opportunities;
5. Always cite sources when presenting research findings;
6. Present data in tables or structured formats when appropriate;
7. Reply in English unless specifically asked for another language.""",
    # Pass Python callables for function calling
    functions=functions_map,
    # Extended metadata for Cosmos DB
    description="Product Research Assistant helps product teams conduct market research, analyze competitors, and gather insights. It streamlines the research process by organizing findings and identifying key opportunities for product development.",
    category="Research & Analytics",
    status="dev",
    avatar_url="product-research-avatar.jpeg",
    function_list=[
        {"function_id": "a1b2c3d4-5678-90ab-cdef-123456789abc",
            "function_name": "toolset_bing"},
        {"function_id": "f9e8d7c6-b5a4-3210-fedc-ba9876543210",
            "function_name": "research_analyzer"}
    ],
    knowledge_base=[
        {"kb_name": "Product Management Best Practices",
            "kb_index": "index-pm-practices"},
        {"kb_name": "Market Research Templates",
            "kb_index": "index-research-templates"}
    ],
    sample_prompts=[
        "What are the latest trends in AI-powered productivity tools?",
        "Analyze the top 3 competitors in the project management software space",
        "What are common user pain points in collaborative document editing?",
        "Research emerging technologies in the DevOps space"
    ],
    scenarios=[
        {"title": "Product Management", "color": "info"},
        {"title": "Market Research", "color": "primary"}
    ],
    maintainers=[
        {
            "photoURL": "6acb666a-2cc7-49ac-ac9b-f4bb28cbbeea.jpg",
            "avatar": "",
            "photo": "",
            "email": "xle@microsoft.com",
            "name": "Lei Xu",
            "permission": "view"
        }
    ]
)

print("\n✅ Agent created successfully")
print(f"📋 Azure Agent ID: {agent.id}")
print(f"📝 Agent Name: {agent.name}")
print(
    f"🛠️  Functions: {len(function_tool.definitions)} (toolset_bing, research_analyzer)")

## Example 2: Retrieve Agent Metadata from Cosmos DB


In [ ]:
# Get agent metadata from Cosmos DB using Azure agent ID
metadata = agent_manager.get_agent_metadata(azure_agent_id=agent.id)

if metadata:
    print("✅ Agent metadata retrieved from Cosmos DB:")
    print(f"  Local ID: {metadata['id']}")
    print(f"  Azure Agent ID: {metadata['azure_agent_id']}")
    print(f"  Name: {metadata['name']}")
    print(f"  Category: {metadata['category']}")
    print(f"  Status: {metadata['status']}")
    print(f"  Functions: {len(metadata.get('functionList', []))}")
    print(f"  Sample Prompts: {len(metadata.get('samplePrompts', []))}")
    print(f"  Scenarios: {metadata.get('scenarios', [])}")
    print(f"  Maintainers: {len(metadata.get('maintainers', []))}")

## Example 3: Update Agent and Metadata


In [ ]:
# Update agent with new metadata
updated_agent = agent_manager.update_agent(
    agent_id=agent.id,
    status="under_review", # New status
    description="Updated Product Research Assistant with enhanced competitor analysis capabilities and market trend tracking.",
    sample_prompts=[
        "What are the latest trends in AI-powered productivity tools?",
        "Analyze the top 3 competitors in the project management software space",
        "What are common user pain points in collaborative document editing?",
        "Research emerging technologies in the DevOps space",
        "What are the pricing strategies of leading SaaS products?"  # New prompt
    ]
)

print(f"✅ Agent updated: {updated_agent.id}")

# Verify update
metadata = agent_manager.get_agent_metadata(azure_agent_id=agent.id)
print("\n📝 Updated metadata:")
print(f"  Status: {metadata['status']}")
print(f"  Description: {metadata['description'][:50]}...")
print(f"  Sample Prompts: {len(metadata['samplePrompts'])}")

## Example 4: Query and Filter Agents


In [ ]:
# List all agents
print("📋 All agents:")
all_agents = agent_manager.list_agents_metadata()
for agent_meta in all_agents:
    print(
        f"  - {agent_meta['name']} ({agent_meta['category']}) - Status: {agent_meta['status']}")

# Filter by status
print("\n📋 Published agents:")
published = agent_manager.list_agents_metadata(status="published")
for agent_meta in published:
    print(f"  - {agent_meta['name']}")

# Filter by category
print("\n📋 Agents in 'Research & Analytics' category:")
research = agent_manager.list_agents_metadata(category="Research & Analytics")
for agent_meta in research:
    print(f"  - {agent_meta['name']}")

## Example 5: Search Agents


In [ ]:
# Search for agents
search_results = agent_manager.search_agents_metadata("product", limit=5)

print("🔍 Search results for 'product':")
for agent_meta in search_results:
    print(f"  - {agent_meta['name']}: {agent_meta['description'][:60]}...")

## Example 6: Database Statistics


In [ ]:
# Get database statistics
stats = agent_manager.get_db_stats()

print("📊 Database Statistics:")
print(f"  Total Agents: {stats.get('total_agents', 0)}")
print("\n  By Status:")
for status, count in stats.get('by_status', {}).items():
    print(f"    - {status}: {count}")
print("\n  By Category:")
for category, count in stats.get('by_category', {}).items():
    print(f"    - {category}: {count}")

## Example 7: Export and Import Agent Metadata


In [ ]:
# Export agent metadata to JSON
if metadata:
    output_file = f"agent_export_{metadata['id']}.json"
    agent_manager.export_agent_metadata(
        agent_id=metadata['id'],
        output_path=output_file
    )
    print(f"📤 Exported to: {output_file}")

    # You can view the exported file
    import json
    with open(output_file, 'r') as f:
        exported_data = json.load(f)

    print("\n📋 Exported data preview:")
    print(f"  ID: {exported_data['id']}")
    print(f"  Name: {exported_data['name']}")
    print(f"  Azure Agent ID: {exported_data['azure_agent_id']}")
    print(f"  Category: {exported_data['category']}")

## Cleanup: Delete All Resources


In [ ]:
# Collect all agent and thread IDs
agent_ids_to_delete = [agent.id]

# Cleanup using the manager
result = agent_manager.cleanup(
    agent_ids=agent_ids_to_delete,
    verbose=True
)

print(f"\n✅ Cleanup summary: {result}")

# Verify cleanup
stats = agent_manager.get_db_stats()
print(f"\n📊 Remaining agents: {stats.get('total_agents', 0)}")

## Summary

### What We Demonstrated:

1. **Extended Metadata**: Created agents with rich metadata (category, scenarios, maintainers, etc.)
2. **Cloud Storage**: Stored agent metadata in Azure Cosmos DB alongside Azure Agent Service
3. **Querying**: Retrieved and filtered agents by status, category
4. **Search**: Full-text search across agent metadata
5. **Updates**: Updated both Azure agents and Cosmos DB metadata
6. **Statistics**: Retrieved database statistics
7. **Export/Import**: Exported and imported agent metadata to/from JSON
8. **Integration**: Agents work seamlessly with conversations and function calling

### Key Benefits:

- **Dual Storage**: Azure Agent Service for runtime, Cosmos DB for rich metadata and querying
- **Global Distribution**: Cosmos DB provides low-latency access worldwide
- **AAD Authentication**: Secure authentication using DefaultAzureCredential
- **Scalability**: Cosmos DB handles large-scale agent metadata storage
- **Searchability**: Fast queries using Cosmos DB SQL API
- **Audit Trail**: Track creation dates, modifications, and maintainers

### Cosmos DB Schema:

The Cosmos DB documents include:

- `id`: Unique agent identifier
- `azure_agent_id`: Azure Agent Service ID
- `name`, `description`, `instruction`: Agent details
- `category`, `status`: Classification and lifecycle
- `functionList`: Agent function definitions (array of strings with "<sep>" delimiter)
- `knowledgeBase`: Knowledge base references (string with "<sep>" delimiter)
- `samplePrompts`: Sample user prompts (array)
- `scenarios`: Scenario tags (array)
- `maintainers`: Maintainer information (array)
- `dateCreated`, `dateModified`, `lastActivity`: Timestamps
